In [1]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from datetime import datetime
from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd

from openpyxl import load_workbook, Workbook
from tkinter import messagebox
from docx import Document
import win32com.client
from tkinter import Tk
import os
import re

In [2]:
class Navegacao():
    def __init__(self):

        servico = Service(ChromeDriverManager().install())

        ask = messagebox.askyesno("Esconder Navegador", "Deseja esconder o navegador?")
        janela = Tk()
        if ask:
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            navegador = webdriver.Chrome(options=options, service=servico)
        else: navegador = webdriver.Chrome(service=servico)
        janela.destroy()
        self.vars = {'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.navegador = navegador

    def esperar_elementos_carregarem(navegador, tipo_elemento: str, nome_elemento: str, varios_elementos: bool = False, tempo_maximo_espera: int = 30):
        """Espera um ou vários elementos carregarem na página

        Args:
            tipo_elemento (str): tipo do elemento
            nome_elemento (str): nome do elemento
            varios_elementos (bool): procurar mais de um elemento
            tempo_maximo_espera (int): tempo determinado para executar a espera, em segundos. Padrão 30 (0,5 min).
        """    
        tempo_inicial = contar() 
        
        if varios_elementos:
            elementos = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_all_elements_located((tipo_elemento, nome_elemento)))
        else:
            elemento = WebDriverWait(navegador, tempo_maximo_espera, ignored_exceptions=(NoSuchElementException,StaleElementReferenceException))\
                            .until(expected_conditions.presence_of_element_located((tipo_elemento, nome_elemento)))
            
        tempo_final = contar()

        if varios_elementos:
            print(f'>>> Elementos "{nome_elemento}" encontrados em {round(tempo_final - tempo_inicial, 2)}s! (len({len(elementos)}))')
            return elementos
        else:
            if len(elemento.text) > 20:
                elemento_texto_resumido = elemento.text[:20]
            else:
                elemento_texto_resumido = elemento.text
                
            print(f'>>> Elemento "{elemento_texto_resumido}" encontrado em {round(tempo_final - tempo_inicial, 2)}s!')
            return elemento
        
    def LISNE(self, NE, afim):
        print('\n=============== Executando LISNE! ===============')
        self.navegadorswitch_to.window(self.navegadorwindow_handles[-1])
        NE = NE.split("NE")[1]
        lisne_link = afim + '/Lisne.do'
        self.navegador.get(lisne_link)

        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.navegador.find_element(By.NAME, 'cboUnidadeGestora').send_keys('270101 - Secretaria Municipal de Infraestrutura')
        self.navegador.find_element(By.ID, 'mkdNumeroNE').clear()
        self.navegador.find_element(By.ID, 'mkdNumeroNE').send_keys(NE)
        self.navegador.find_element(By.ID, 'btnProcurar').click()

        self.esperar_elementos_carregarem(self.navegador, By.CSS_SELECTOR, '[class="btn btn-outline-primary waves-effect waves-light"]').click()

        para(2)
        # navegador.switch_to.window(navegador.window_handles[-1])

        fonte_de_recurso = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtFonte"]').get_attribute('value')
        natureza_despesa = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNaturezaDespesa"]').get_attribute('value')
        empenho_original = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtNEOriginal"]').get_attribute('value')
        processo = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtProcesso"]').get_attribute('value')
        valor = self.navegador.find_element(By.CSS_SELECTOR, '[ng-model="notaDeEmpenho.strTxtValor"]').get_attribute('value')

        self.navegador.find_element(By.CSS_SELECTOR, '[data-dismiss="modal-empenho"]').click()

        return (NE, fonte_de_recurso, natureza_despesa, empenho_original, processo, valor)

In [10]:
def ler_arquivo_docx(caminho_arquivo):
    print('Executando ler_arquivo_docx!')
    doc = Document(caminho_arquivo)
    
    conteudo = ""
    for paragrafo in doc.paragraphs:
        conteudo += paragrafo.text + "\n"
    
    return conteudo

def ler_arquivo_doc(caminho_arquivo_doc, word):
    print('Executando ler_arquivo_doc!')
    doc = word.Documents.Open(caminho_arquivo_doc)
    
    # Extrai o texto do documento .doc
    texto = doc.Content.Text
    
    # Fecha o documento e o aplicativo Word
    doc.Close()
    
    return texto

def extraindo_informacoes(texto):
    print(texto.replace('\r','\n'))
    print('Executando extraindo_informacoes!')
    # Extrair número do contrato
    numero_contrato = re.search(r"Contrato nº\.\s(\d+)", texto).group(1)

    # Extrair data do contrato
    data_contrato = re.search(r"celebrado em\s?(\d{2}\.\d{2}\.\d{4})", texto).group(1)
    data_contrato = datetime.strptime(data_contrato, "%d.%m.%Y")

    # Extrair nome da empresa
    nome_empresa = re.search(r"empresa\s+(.+)\.", texto).group(1)

    # Extrair objeto/obra
    objeto_obra = re.search(r"OBJETO:\s(.+), [de acordo|conforme]", texto).group(1)

    # Extrair valor global
    valor_global = re.search(r"VALOR GLOBAL:\sR\$ ([\d.,]+)", texto).group(1)

    # Extrair nota de empenho
    nota_empenho = re.findall(r"[\.|\s](\d+)", texto)
    'Empenhos nº. 01136 e nº 01138,'

    # Extrair prazo
    prazo = re.search(r"PRAZO:\D+(\d+)\D+", texto).group(1)

    # Imprimir as informações extraídas
    print("Número do contrato:", numero_contrato)
    print("Data do contrato:", data_contrato)
    print("Empresa:", nome_empresa)
    print("Objeto/Obra:", objeto_obra)
    print("Valor global:", valor_global)
    print("Nota de empenho:", nota_empenho)
    print("Prazo:", prazo)

    return (numero_contrato, data_contrato, nome_empresa, objeto_obra, valor_global, nota_empenho, prazo)

def ler_arquivos_em_pasta(caminho_pasta):
    print('Extraindo ler_arquivos_em_pasta!')
    df = {}
    arquivos = os.listdir(caminho_pasta)
    word = win32com.client.Dispatch("Word.Application")
    
    # print('Extraindo ler_arquivos_em_pasta!')
    for arquivo in arquivos:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        
        if '.doc' in arquivo and os.path.isfile(caminho_arquivo): texto = ler_arquivo_doc(caminho_arquivo_doc=caminho_arquivo, word=word)
        else: texto = ler_arquivo_docx(caminho_arquivo=caminho_arquivo)

        informacao = extraindo_informacoes(texto)
        df[arquivo] = informacao
    
    word.Quit()

    return df

# Exemplo de uso
caminho_pasta = r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\CONTRATO_2014"
df = ler_arquivos_em_pasta(caminho_pasta)
# print(d)

Extraindo ler_arquivos_em_pasta!
Executando ler_arquivo_doc!
EXTRATO


1.ESPÉCIE E DATA: Contrato nº. 026, celebrado em 01.07.2014.
2.PARTES: Município de Manaus (SEMINF) e a empresa CONSTRUTORA SÃO FRANCISCO LTDA.
3.OBJETO: Locação de Veículos Pesados (Caminhões e outros) e Equipamentos de Pequeno e Grande Porte – Lote I, conforme o Pregão nº. 002/2013-CLS/PM – Registros de Preços e quantitativos registrados na Ata de Registro de Preços nº. 002/2013-SEMINF
4.VALOR GLOBAL: R$ 8.000.000,00 (oito milhões de reais).
5.DOTAÇÃO ORÇAMENTÁRIA: Empenhos nº. 01136 e nº 01138, datados de 30.06.2014, à conta da seguinte rubrica orçamentária: 27100 -  15.451.1060.2243.0000 – 33903982 – Fontes nº. 02240084 e nº. 01000000.
6.PRAZO: O prazo de vigência do presente contrato será de 05 (cinco) meses, a contar da data de sua assinatura.

Manaus, 01 de julho de 2014.



ORLANDO CABRAL HOLANDA
SUBSECRETÁRIO MUNICIPAL DE OBRAS PÚBLICAS
SEMINF



Executando extraindo_informacoes!
Número do contrato: 026
Dat

AttributeError: 'NoneType' object has no attribute 'group'

In [9]:
re.findall(r"[\.|\s](\d+)","Nota de Empenho nº.00935, n°.00547, nº 00541, n° 00542, nº. 00900 e n°. 00507")

['00935', '00547', '00541', '00542', '00900', '00507']

In [ ]:
driver = Navegacao()
driver.navegador.get('https://www.google.com/')
driver.navegador.close()